In [ ]:
import numpy as np

def choose_portfolio(agent, utility_variables, current_t, model_parameters, map_parameters, demographic_variables, map_variables):
    # as of now, the agent has not decided to move
    moved = []

    # choose a set of locations to evaluate - the current location, some other good locations from past searches,
    # and a few random new ones
    current_location = agent.matrix_location
    sorted_locations = np.argsort(agent.best_portfolio_values)[::-1]
    sorted_index = sorted_locations[sorted_locations > 0]
    best_locations = sorted_index[:min(len(sorted_index), agent.num_best_location)]
    other_random_locations = np.where(np.any(agent.knows_income_location, axis=1))[0]
    random_locations = np.random.choice(
        other_random_locations, size=min(len(other_random_locations), agent.num_random_location), replace=False
    )
    location_list = np.concatenate(([current_location], best_locations, random_locations))

    # remove duplicates
    location_list = np.unique(location_list)

    # make note of the total number of portfolios we expect to evaluate in each location
    total_num_portfolios = agent.num_best_portfolio + agent.num_random_portfolio

    # since we will try to match historical data to the appropriate season, create labels for what part of the cycle
    # our historical data and our evaluation period are
    period_in_cycle_data = np.arange(1, current_t + 1) % model_parameters.cycle_length
    period_in_cycle_evaluation = np.arange(agent.num_periods_evaluate) % model_parameters.cycle_length

    # set up the discount and risk factors
    discount_age_scaling = np.interp(
        agent.age, demographic_variables.age_points_pref, demographic_variables.age_discount_rate_factor
    )
    discount_factor = 1 / ((1 + agent.discount_rate * discount_age_scaling) ** np.arange(agent.num_periods_evaluate))
    risk_coeff = 1 - agent.r_value

    # initialize vectors to hold the best-case portfolios and values for each location
    location_value = np.full(len(location_list), np.nan)
    location_portfolio = [None] * len(location_list)
    location_aspiration = [None] * len(location_list)
    location_fidelity = [None] * len(location_list)
    location_access_codes = [None] * len(location_list)
    location_moving_costs = np.zeros(len(location_list))
    considered_portfolio_set = []  # List of portfolios considered by agent across all locations in this time step

    # Check which layers are "selectable" based on agent prereqs
    selectable = selectable_flag(
        utility_variables.utility_prereqs,
        utility_variables.utility_access_codes_mat,
        utility_variables.utility_access_costs,
        agent.income_layers_history,
        agent.wealth,
        current_t,
    )

    # for each location, find a good income portfolio - the current portfolio (if this is the home city),
    # some other good portfolios from past searches, and a few random new ones
    for index_l in range(len(location_list)):
        # initialize a few things - a blank logical matrix to hold the layers active in each portfolio,
        # and the moving cost that will apply to every portfolio in this location
        portfolio_set = np.zeros((total_num_portfolios, utility_variables.utility_history.shape[2]), dtype=bool)
        aspiration_set = np.zeros((total_num_portfolios, utility_variables.utility_history.shape[2]), dtype=bool)
        fidelity_set = np.zeros(total_num_portfolios)
        current_portfolio = 0
        current_moving_cost = map_variables.moving_costs[current_location, location_list[index_l]]
        location_moving_costs[index
